In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [ ]:
import cv2
import numpy as np
from PIL import Image
import horizon.transforms as T

ir16 = cv2.imread(
    "/mnt/fiftyoneDB/Database/Image_Data/Thermal_Images_16Bit/Trip_310_Seq_153/23714953_r.png",
    cv2.IMREAD_UNCHANGED,
)
transform = T.horizon_augment_ir16bit(imgsz=640, im_compression_prob=0.9)
im1 = (transform(image=ir16)["image"].numpy().transpose(1, 2, 0) * 255).astype(np.uint8)

transform = T.horizon_base_ir16bit(imgsz=640)
im2 = (transform(image=ir16)["image"].numpy().transpose(1, 2, 0) * 255).astype(np.uint8)

rgb = cv2.cvtColor(cv2.imread(
    "/mnt/fiftyoneDB/Database/Image_Data/RGB_Images/Trip_310_Seq_153/23714953.jpg",
), cv2.COLOR_BGR2RGB)
transform = T.horizon_augment_rgb(imgsz=640, im_compression_prob=0.9)
im3 = (transform(image=rgb)["image"].numpy().transpose(1, 2, 0) * 255).astype(np.uint8)

transform = T.horizon_base_rgb(imgsz=640)
im4 = (transform(image=rgb)["image"].numpy().transpose(1, 2, 0) * 255).astype(np.uint8)

# stack in 2x2 grid
im = np.vstack([np.hstack([im1, im2]), np.hstack([im3, im4])])
Image.fromarray(im).reduce(2)


In [6]:
from seavision.improc16 import DefaultThermalTransform

ir8 = DefaultThermalTransform()(ir16)

In [ ]:
import albumentations as A
import utils.albumextensions as Ax

transform = A.Compose(
    [
        A.RandomCropFromBorders(p=0.1),
        Ax.ResizeIfNeeded(max_size=640),
        A.HorizontalFlip(p=0.5),
        A.PadIfNeeded(
            min_height=640, min_width=640, border_mode=cv2.BORDER_CONSTANT, value=0
        ),
        # A.RandomScale(scale_limit=0.25, p=1),
        A.Affine(
            p=0.75,
            scale=(0.8, 1.1),
            rotate=(-20, 20),
            translate_percent=(-0.1, 0.1),
            balanced_scale=True,
            keep_ratio=True,
            fit_output=False,
            mode=cv2.BORDER_CONSTANT,
            cval=0,
        ),
        # A.RandomResizedCrop(p=1, size=(640, 640), ratio=(1,1), scale=(0.5, 1.0)),
    ]
)
transformed = transform(image=ir8)["image"]
print(transformed.shape, f"aspect ratio: {transformed.shape[1] / transformed.shape[0]}")
Image.fromarray(transformed)